In [ ]:
import requests
from requests.auth import HTTPDigestAuth
from xml.etree import ElementTree
import json
from ipython_secrets import get_secret

In [ ]:
OPENCAST_BASE_URL = "http://admin.prod-oc.dcex.harvard.edu"
OPENCAST_API_USER = get_secret("OPENCAST_API_USER", prompt="Opencast API User")
OPENCAST_API_PASS = get_secret("OPENCAST_API_PASS", prompt="Opencast API Password")

In [ ]:
http_auth = HTTPDigestAuth(OPENCAST_API_USER, OPENCAST_API_PASS)
opencast_headers = {
    'X-REQUESTED-AUTH': 'Digest',
    'X-Opencast-Matterhorn-Authorization': 'true'
}

# Example

This example fetches one workflow's data and calculates the overall performance rating and the rating of each operation within the workflow based on duration and the length of the video.

In [ ]:
wf_id = "33657"
wf_url = OPENCAST_BASE_URL + "/workflow/instance/{id}.json".format(id=wf_id)
wf_resp = requests.get(wf_url, auth=http_auth, headers=opencast_headers)
wf_data = wf_resp.json()['workflow']
wf_data

In [ ]:
mediapackage_id = wf_data['mediapackage']['id']
assets_url = OPENCAST_BASE_URL + "/assets/episode/{mpid}".format(mpid=mediapackage_id)
assets_resp = requests.get(assets_url, auth=http_auth, headers=opencast_headers)
assets_xml = ElementTree.fromstring(assets_resp.content)

In [ ]:
# https://docs.python.org/3.6/library/xml.etree.elementtree.html#parsing-xml-with-namespaces
ns = {'oc': 'http://mediapackage.opencastproject.org'}

video_lengths = {}
for track in assets_xml.findall('.//oc:track', ns):
    type = track.attrib['type']
    for track_type in ['source', 'delivery']:
        if type.endswith(track_type) and track_type not in video_lengths:
            video_lengths[track_type] = int(track.find('oc:duration', ns).text)
video_lengths        

In [ ]:
wf_id = wf_data['id']
wf_state = wf_data['state']
wf_ops = wf_data['operations']['operation']
completed_ops = [op for op in wf_ops if 'completed' in op]

if len(completed_ops) and wf_state in ['SUCCEEDED', 'FAILED']:
    first_op = completed_ops[0]
    last_op = completed_ops[-1]
    wf_duration = int((last_op['completed'] - first_op['completed']))
    wf_perf_metric = round(float(wf_duration) / video_lengths['delivery'], 3)
else:
    wf_duration = None
    wf_perf_metric = None
    
print("workflow duration: " + str(wf_duration))
print("workflow performance metric: " + str(wf_perf_metric))

In [ ]:
for op in completed_ops:
    op_type = op['id']
    op_duration = int(op['completed'] - op['started'])
    op_perf_metric = round(float(op_duration) / video_lengths['delivery'], 3)
    print("{},{},{},{}".format(wf_id, op_type, op_duration, op_perf_metric))